In [23]:
# Because fetch_mldata is deprecated since scikit-learn 0.20, we'll use fetch_openml instead.

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

pd.set_option('display.max_columns', 500)

In [4]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

In [10]:
print(dir(mnist))
len(mnist['target']), type(mnist['target']), len(mnist['data']), type(mnist['data'])

['DESCR', 'categories', 'data', 'details', 'feature_names', 'frame', 'target', 'target_names', 'url']


(70000, pandas.core.series.Series, 70000, pandas.core.frame.DataFrame)

In [34]:
print(mnist['url'])
print()

pd.set_option('display.max_columns', 18)
pd.set_option('display.width', 300)
print(mnist['data'].tail())
print()

print(mnist['target_names'])
print(mnist['target'].tail())
print()
print(mnist['DESCR'])

https://www.openml.org/d/554

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  ...  pixel776  pixel777  pixel778  pixel779  pixel780  pixel781  pixel782  pixel783  pixel784
69995     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
69996     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
69997     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
69998     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0
69999     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  ...       0.0       0.0   

In [12]:
print(mnist['target'].head())

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']


In [13]:
# fetch_openml() returns targets as strings
mnist["target"] = mnist['target'].astype(np.int8)

In [14]:
print(mnist['target'].head())

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: int8
